In [1]:
import pandas as pd
import scipy.sparse as sp
from tqdm import tqdm
import numpy as np

In [2]:
df = pd.read_csv('crowd_labels.csv')
df = df[['left', 'right', 'label']]

## Adjacency matrix

In [3]:
# get all unique values in df['left'] and df['right']
persons = list(set(df['left'].unique().tolist() + df['right'].unique().tolist()))

In [4]:
adj = sp.lil_matrix((len(persons), len(persons)))

for row in tqdm(df.itertuples()):
    label = (row.label == row.left)  # 1 if left is older, 0 if right is older
    label = 2 * label - 1
    idx_left = persons.index(row.left)
    idx_right = persons.index(row.right)
    adj[idx_left, idx_right] = label
    # adj[idx_right, idx_left] = -label

# save adjacency matrix as a sparse matrix
adj = adj.tocsr()
sp.save_npz(f'./adj.npz', adj)

250249it [01:19, 3158.50it/s]


In [5]:
adj

<9150x9150 sparse matrix of type '<class 'numpy.float64'>'
	with 250249 stored elements in Compressed Sparse Row format>

## Ground truth

In [6]:
gt = pd.read_csv('gt.csv')

In [7]:
y = np.zeros(len(persons))
for row in tqdm(gt.itertuples()):
    idx = persons.index(row.label)
    y[idx] = row.score
# save as `y.npy`
np.save('./y.npy', y)

9150it [00:01, 7025.69it/s]
